<a href="https://colab.research.google.com/github/deepakri201/monai_full_ct_segmentation_bundle/blob/main/MONAI_label_app_to_bundle_full_ct_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script converts a MONAI label app for full CT segmentation to a bundle. 

Deepa Krishnaswamy
Brigham and Women's Hospital 
Jan 2023

# Install and then restart runtime

In [1]:
# !pip install monailabel -U
!pip install monailabel-weekly


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 KB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 11.7 MB/s eta 0:00:00
  Preparing metadata 

# Environment setup

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# !pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==0.6.0"
# !pip install monai-weekly
!pip install 'monai[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 18.5 MB/s eta 0:00:00


In [4]:
import os 
import sys 

In [5]:

import tempfile
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable

import monai
from monai.config import print_config
from monai.utils import first
from monai.config import KeysCollection
from monai.data import Dataset, ArrayDataset, create_test_image_3d, DataLoader
# from monai.transforms import (
#     Transform,
#     MapTransform,
#     Randomizable,
#     AddChannel,
#     AddChanneld,
#     Compose,
#     LoadImage,
#     LoadImaged,
#     Lambda,
#     Lambdad,
#     RandSpatialCrop,
#     RandSpatialCropd,
#     ToTensor,
#     ToTensord,
#     Orientation, 
#     Rotate
# )
print_config()

from monai.bundle import ConfigParser
import json 

import numpy 
import torch

MONAI version: 1.1.0
Numpy version: 1.21.6
Pytorch version: 1.13.1+cu116
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /usr/local/lib/python3.8/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.10
Nibabel version: 3.0.2
scikit-image version: 0.18.3
Pillow version: 9.4.0
Tensorboard version: 2.9.1
gdown version: 4.4.0
TorchVision version: 0.14.1+cu116
tqdm version: 4.64.1
lmdb version: 0.99
psutil version: 5.4.8
pandas version: 1.3.5
einops version: 0.6.0
transformers version: 4.21.3
mlflow version: 2.1.1
pynrrd version: 0.4.3

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [6]:
import monailabel

if not os.path.isdir('/content/apps'):
  os.mkdir('/content/apps')

!monailabel --help
# !monailabel apps
# !monailabel apps --download --name radiology --output apps
!monailabel apps --name radiology --download --output '/content/apps'

# https://docs.monai.io/projects/label/en/latest/quickstart.html 

Using PYTHONPATH=/usr:/env/python

usage: monailabel
       [-h]
       [-v]
       {start_server,apps,datasets,plugins}
       ...

positional arguments:
  {start_server,apps,datasets,plugins}
    sub-command
    help
    start_server
    Start
    Application
    Server
    apps
    list or
    download
    sample apps
    datasets
    list or
    download
    sample
    datasets
    plugins
    list or
    download
    viewer
    plugins

optional arguments:
  -h, --help
    show this
    help
    message and
    exit
  -v, --version
    print
    version
Using PYTHONPATH=/usr:/env/python

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.8/dist-packages/monailabel/main.py", line 342, in <module>
    Main().run()
  File "/usr/local/lib/python3.8/dist-packages/monai

In [7]:
# Clone the MONAI Label App directory so I can get some fields I need 

!git clone https://github.com/Project-MONAI/MONAILabel.git

Cloning into 'MONAILabel'...
remote: Enumerating objects: 12311, done.
remote: Counting objects: 100% (579/579), done.
remote: Compressing objects: 100% (358/358), done.
remote: Total 12311 (delta 305), reused 430 (delta 211), pack-reused 11732
Receiving objects: 100% (12311/12311), 46.69 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (8437/8437), done.


# Parameterization

In [8]:
project_id = 'idc-external-018'
location_id = 'us-central1'
# monai_bucket = os.path.join('gs://', 'monai','verse','test')
monai_bucket = os.path.join('gs://', 'monai', 'spleen', 'test')
print(monai_bucket)


gs://monai/spleen/test


# Download the spleen bundle to see examples of the json file

In [9]:
# Download the spleen monai bundle so I can see an example of the json files and directory structure --> get latest? check differences. 
# !python -m monai.bundle download --name spleen_ct_segmentation --version "0.1.1" --bundle_dir "./"
!python -m monai.bundle download --name spleen_ct_segmentation --version "0.3.7" --bundle_dir "./"

2023-02-02 10:40:24,737 - INFO - --- input summary of monai.bundle.scripts.download ---
2023-02-02 10:40:24,737 - INFO - > name: 'spleen_ct_segmentation'
2023-02-02 10:40:24,737 - INFO - > version: '0.3.7'
2023-02-02 10:40:24,737 - INFO - > bundle_dir: './'
2023-02-02 10:40:24,737 - INFO - > source: 'github'
2023-02-02 10:40:24,737 - INFO - > remove_prefix: 'monai_'
2023-02-02 10:40:24,737 - INFO - > progress: True
2023-02-02 10:40:24,737 - INFO - ---


spleen_ct_segmentation_v0.3.7.zip: 33.9MB [00:04, 7.71MB/s]                
2023-02-02 10:40:29,356 - INFO - Downloaded: spleen_ct_segmentation_v0.3.7.zip
2023-02-02 10:40:29,356 - INFO - Expected md5 is None, skip md5 check for file spleen_ct_segmentation_v0.3.7.zip.
2023-02-02 10:40:29,356 - INFO - Writing into directory: ..


# Set up directories for full_ct_segmentation

In [10]:
# Setup of directories for full_ct_segmentation 

main_dir = "/content/full_ct_segmentation"
config_dir = os.path.join(main_dir, 'configs')
model_dir = os.path.join(main_dir, 'models')
data_dir = os.path.join("/content/data")
data_dir_Ts = os.path.join(data_dir, "imagesTs")

if not os.path.isdir(main_dir):
  os.mkdir(main_dir)
if not os.path.isdir(config_dir):
  os.mkdir(config_dir)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)
if not os.path.isdir(data_dir):
  os.mkdir(data_dir)
if not os.path.isdir(data_dir_Ts):
  os.mkdir(data_dir_Ts)

# Get the model from Andres' google drive folder

In [11]:
# https://drive.google.com/drive/folders/12Z2XjrDCGhWGRvJZj82n28W5siLcWC2q?usp=share_link

In [12]:
# !wget "https://drive.google.com/drive/folders/12Z2XjrDCGhWGRvJZj82n28W5siLcWC2q?usp=share_link" "/content/andres"
# Get from github for now 

# model_path_github = "https://github.com/deepakri201/monai_app_to_bundle/tree/main/pretrained_models/segmentation_full_ct.pt"
model_path_github = "https://raw.githubusercontent.com/deepakri201/monai_app_to_bundle/main/pretrained_models/segmentation_full_ct.pt"
model_path_download = os.path.join(model_dir, "model.pt")
!wget -O $model_path_download $model_path_github 

# !wget $model_path_github 
# !wget -O $query_download_path https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/common/queries/NSCLC_Radiomics_query.txt

# !wget $model_path_github 
# !mv "/content/radiology_segmentation_segresnet_localization_spine.pt" $model_path

--2023-02-02 10:40:45--  https://raw.githubusercontent.com/deepakri201/monai_app_to_bundle/main/pretrained_models/segmentation_full_ct.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75224450 (72M) [application/octet-stream]
Saving to: ‘/content/full_ct_segmentation/models/model.pt’

/content/full_ct_se 100%[===================>]  71.74M   123MB/s    in 0.6s    

2023-02-02 10:40:51 (123 MB/s) - ‘/content/full_ct_segmentation/models/model.pt’ saved [75224450/75224450]



# Copy the logging conf file from the spleen bundle for now 

In [13]:
# Copy the logging.conf file for now 
!cp "/content/spleen_ct_segmentation/configs/logging.conf" "/content/full_ct_segmentation/configs/logging.conf"


# Download the spleen data for inference - data already in bucket

In [14]:
# Instead download from a bucket for now 

!gsutil -m cp -r $monai_bucket/*.nii.gz $data_dir_Ts


Copying gs://monai/spleen/test/spleen_3.nii.gz...
| [1/1 files][ 10.8 MiB/ 10.8 MiB] 100% Done                                    
Operation completed over 1 objects/10.8 MiB.                                     


# Create the metajson file

In [20]:

metajson_dict = {
    "schema": "https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/meta_schema_20220324.json",
    # "version": "0.1.1",
    # "changelog": {
    #     "0.1.1": "disable image saving during evaluation",
    #     "0.1.0": "complete the model package",
    #     "0.0.1": "initialize the model package structure"
    # },
    "monai_version": "0.9.0",
    "pytorch_version": "1.10.0",
    "numpy_version": "1.21.2",
    # "optional_packages_version": {
    #     "nibabel": "3.2.1",
    #     "pytorch-ignite": "0.4.8"
    # },
    "task": "Decathlon spleen segmentation",
    "description": "A pre-trained model for volumetric (3D) segmentation of the spleen from CT image",
    "authors": "Deepa Krishnaswamy adapted from Andres Diaz-Pinto full ct segmentation label app code",
    "copyright": "Copyright (c) MONAI Consortium",
    "data_source": "TotalSegmentator data",
    "data_type": "nibabel",
    "image_classes": "single channel data, intensity scaled to [0, 1]",
    "label_classes": "25 channel data, 1-24 vertebrae and 0 is everything else",
    "pred_classes": "25 channels OneHot data, channel 1-24 vertebra, channel 0 is background",
    "eval_metrics": {
        "mean_dice": 0.96
    },
    "intended_use": "This is an example, not to be used for diagnostic purposes",
    # "references": [
    #     "Xia, Yingda, et al. '3D Semi-Supervised Learning with Uncertainty-Aware Multi-View Co-Training. arXiv preprint arXiv:1811.12506 (2018). https://arxiv.org/abs/1811.12506.",
    #     "Kerfoot E., Clough J., Oksuz I., Lee J., King A.P., Schnabel J.A. (2019) Left-Ventricle Quantification Using Residual U-Net. In: Pop M. et al. (eds) Statistical Atlases and Computational Models of the Heart. Atrial Segmentation and LV Quantification Challenges. STACOM 2018. Lecture Notes in Computer Science, vol 11395. Springer, Cham. https://doi.org/10.1007/978-3-030-12029-0_40"
    # ],
    "network_data_format": {
        "inputs": {
            "image": {
                "type": "image",
                "format": "hounsfield",
                "modality": "CT",
                "num_channels": 1, # check this
                "spatial_shape": [
                    96,
                    96,
                    96
                ],
                "dtype": "float32",
                "value_range": [
                    0,
                    1
                ],
                "is_patch_data": True,
                "channel_def": {
                    "0": "image"
                }
            }
        },
        "outputs": {
            "pred": {
                "type": "image",
                "format": "segmentation",
                "num_channels": 104, # can get this automatically - check if 104 to 105 including background
                # "num_channels": 2, 
                "spatial_shape": [
                    96,
                    96,
                    96
                ],
                "dtype": "float32",
                "value_range": [
                    0,
                    1
                ],
                "is_patch_data": True,
                # "channel_def": { # WHY 25. should be 1 for the first spine localization step? 
                #     "0": "background",
                #     "1": "spine"
                # }
                "channel_def": {
                "spleen": 1,
                "kidney_right": 2,
                "kidney_left": 3,
                "gallbladder": 4,
                "liver": 5,
                "stomach": 6,
                "aorta": 7,
                "inferior_vena_cava": 8,
                "portal_vein_and_splenic_vein": 9,
                "pancreas": 10,
                "adrenal_gland_right": 11,
                "adrenal_gland_left": 12,
                "lung_upper_lobe_left": 13,
                "lung_lower_lobe_left": 14,
                "lung_upper_lobe_right": 15,
                "lung_middle_lobe_right": 16,
                "lung_lower_lobe_right": 17,
                "vertebrae_L5": 18,
                "vertebrae_L4": 19,
                "vertebrae_L3": 20,
                "vertebrae_L2": 21,
                "vertebrae_L1": 22,
                "vertebrae_T12": 23,
                "vertebrae_T11": 24,
                "vertebrae_T10": 25,
                "vertebrae_T9": 26,
                "vertebrae_T8": 27,
                "vertebrae_T7": 28,
                "vertebrae_T6": 29,
                "vertebrae_T5": 30,
                "vertebrae_T4": 31,
                "vertebrae_T3": 32,
                "vertebrae_T2": 33,
                "vertebrae_T1": 34,
                "vertebrae_C7": 35,
                "vertebrae_C6": 36,
                "vertebrae_C5": 37,
                "vertebrae_C4": 38,
                "vertebrae_C3": 39,
                "vertebrae_C2": 40,
                "vertebrae_C1": 41,
                "esophagus": 42,
                "trachea": 43,
                "heart_myocardium": 44,
                "heart_atrium_left": 45,
                "heart_ventricle_left": 46,
                "heart_atrium_right": 47,
                "heart_ventricle_right": 48,
                "pulmonary_artery": 49,
                "brain": 50,
                "iliac_artery_left": 51,
                "iliac_artery_right": 52,
                "iliac_vena_left": 53,
                "iliac_vena_right": 54,
                "small_bowel": 55,
                "duodenum": 56,
                "colon": 57,
                "rib_left_1": 58,
                "rib_left_2": 59,
                "rib_left_3": 60,
                "rib_left_4": 61,
                "rib_left_5": 62,
                "rib_left_6": 63,
                "rib_left_7": 64,
                "rib_left_8": 65,
                "rib_left_9": 66,
                "rib_left_10": 67,
                "rib_left_11": 68,
                "rib_left_12": 69,
                "rib_right_1": 70,
                "rib_right_2": 71,
                "rib_right_3": 72,
                "rib_right_4": 73,
                "rib_right_5": 74,
                "rib_right_6": 75,
                "rib_right_7": 76,
                "rib_right_8": 77,
                "rib_right_9": 78,
                "rib_right_10": 79,
                "rib_right_11": 80,
                "rib_right_12": 81,
                "humerus_left": 82,
                "humerus_right": 83,
                "scapula_left": 84,
                "scapula_right": 85,
                "clavicula_left": 86,
                "clavicula_right": 87,
                "femur_left": 88,
                "femur_right": 89,
                "hip_left": 90,
                "hip_right": 91,
                "sacrum": 92,
                "face": 93,
                "gluteus_maximus_left": 94,
                "gluteus_maximus_right": 95,
                "gluteus_medius_left": 96,
                "gluteus_medius_right": 97,
                "gluteus_minimus_left": 98,
                "gluteus_minimus_right": 99,
                "autochthon_left": 100,
                "autochthon_right": 101,
                "iliopsoas_left": 102,
                "iliopsoas_right": 103,
                "urinary_bladder": 104,
        }
            }
        }
    }
}

In [18]:
monai_version = monai.__version__
pytorch_version = torch.__version__
numpy_version = numpy.__version__

In [21]:
config = ConfigParser()
metajson_filename = os.path.join(config_dir, 'metadata.json')
# config.export_config_file(metajson_dict, metajson_filename, fmt="json")

metajson_dict["monai_version"] = monai_version 
metajson_dict["pytorch_version"] = pytorch_version
metajson_dict["numpy_version"] = numpy_version

with open(metajson_filename, "w") as outfile: 
  json.dump(metajson_dict, outfile, indent=4)
outfile.close()

# Create the inference json file

In [22]:

inference_dict = {
    "imports": [
        "$import glob",
        "$import os",
        "$import sys",
        # "$from monailabel.transform.post import Restored"
    ],
    "bundle_root": "", 
    "output_dir": "$@bundle_root + '/eval'",
    "dataset_dir": "",
    "datalist": "$list(sorted(glob.glob(@dataset_dir + '/imagesTs/*.nii.gz')))",
    "device": "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
    # "device":"$torch.device('cpu')",
    ### I modified the network_def below ### 
    "network_def": { # https://github.com/Project-MONAI/MONAILabel/blob/main/sample-apps/radiology/lib/configs/localization_spine.py
        "_target_": "SegResNet",
        "spatial_dims": 3,
        "init_filters": 32,
        "in_channels": 1,
        "out_channels": 105,  # can get this automatically later 
        # "out_channels": 2,
        "dropout_prob": 0.2,
        "blocks_down": (1, 2, 2, 4),
        "blocks_up": (1, 1, 1)
    },


    "network": "$@network_def.to(@device)",
    ### I modified the preprocessing below ### # pre_transforms from https://github.com/Project-MONAI/MONAILabel/blob/main/sample-apps/radiology/lib/infers/localization_spine.py 
    "preprocessing": {
        "_target_": "Compose",
        "transforms": [
            {
                "_target_": "LoadImaged", # LoadImaged(keys="image"),
                "keys": "image"
            },
            {
                "_target_": "EnsureTyped", # EnsureTyped(keys="image", device=data.get("device") if data else None),
                "keys": "image"
            },
            {
                "_target_": "EnsureChannelFirstd", #  EnsureChannelFirstd(keys="image"),
                "keys": "image"
            },
            # {
            #     "_target_": "Orientationd", # Orientationd(keys="image", axcodes="RAS"),
            #     "keys": "image",
            #     "axcodes": "RAS"
            # },
            {
                "_target_": "Spacingd", # Spacingd(keys="image", pixdim=self.target_spacing, allow_missing_keys=True),
                "keys": "image",
                "pixdim": [
                    1.5,
                    1.5,
                    1.5
                ],
                "mode": "bilinear"
            },
            {
                "_target_": "NormalizeIntensityd", # NormalizeIntensityd(keys="image", nonzero=True),
                "keys": "image"
            },
            # ScaleIntensityRanged(keys="image", a_min=-1000, a_max=1900, b_min=0.0, b_max=1.0, clip=True),
            {
                "_target_": "GaussianSmoothd", # GaussianSmoothd(keys="image", sigma=0.4),
                "keys": "image",
                "sigma": 0.4 
            },
            {
                "_target_": "ScaleIntensityd", # ScaleIntensityd(keys="image", minv=-1.0, maxv=1.0),
                "keys": "image",
                "minv": -1.0, 
                "maxv": 1.0
            }, 
        ]

},
    "dataset": {
        "_target_": "Dataset",
        "data": "$[{'image': i} for i in @datalist]",
        "transform": "@preprocessing"
    },
    ### Will need to find the batch_size and num_workers ### 
    "dataloader": {
        "_target_": "DataLoader",
        "dataset": "@dataset",
        "batch_size": 1,  # CHECK THIS!
        "shuffle": False,
        "num_workers": 4   # CHECK THIS!
    },
    ### I modified the inferer below ### inferer from https://github.com/Project-MONAI/MONAILabel/blob/main/sample-apps/radiology/lib/infers/localization_spine.py 
    "inferer": {
        "_target_": "SlidingWindowInferer", # SlidingWindowInferer(roi_size=self.roi_size, sw_batch_size=2, overlap=0.4, padding_mode="replicate", mode="gaussian")
        "roi_size": [ # roi_size from here # https://github.com/Project-MONAI/MONAILabel/blob/main/sample-apps/radiology/lib/configs/localization_spine.py
            96,
            96,
            96
        ],
        "sw_batch_size": 2,
        "overlap": 0.4,
        "padding_mode": "replicate", 
        "mode": "gaussian"
    },
    ### Need to modify the below ### post_transforms from https://github.com/Project-MONAI/MONAILabel/blob/main/sample-apps/radiology/lib/infers/localization_spine.py 
    "postprocessing": {
        "_target_": "Compose",
        "transforms": [
           {
                "_target_": "EnsureTyped", # EnsureTyped(keys="pred", device=torch.device("cpu")), # Use this in case of using GPU smaller than 24GB
                "keys": "image",
                # "device": "$torch.device('cpu')",
                # "device": "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')"
            },
           {
                "_target_": "Activationsd", # Activationsd(keys="pred", softmax=True), # https://github.com/Project-MONAI/MONAI/blob/dev/monai/transforms/post/dictionary.py - do I need to specify the other keys? keep defaults.
                "keys": "pred",
                "softmax": True
            },          
           {
                "_target_": "AsDiscreted", # AsDiscreted(keys="pred", argmax=True), # https://github.com/Project-MONAI/MONAI/blob/dev/monai/transforms/post/dictionary.py - do I need to specify the other keys? keep defaults.
                "keys": "pred",
                "argmax": True
            },
            # {
            #     "_target_": "Invertd",
            #     "keys": "pred",
            #     "transform": "@preprocessing",
            #     "orig_keys": "image",
            #     "meta_key_postfix": "meta_dict",
            #     "nearest_interp": False,
            #     "to_tensor": True
            # },
            {
                "_target_": "KeepLargestConnectedComponentd", # KeepLargestConnectedComponentd(keys="pred"), # https://github.com/Project-MONAI/MONAI/blob/dev/monai/transforms/post/dictionary.py  - do I need to specify the other keys? keep defaults.
                "keys": "pred"
            },
            ### Restored is from monailabel - from monailabel.transform.post import Restored ### 
            # {
            #     "_target_": "Restored", # Restored(keys="pred", ref_image="image"), # https://github.com/Project-MONAI/MONAILabel/blob/main/monailabel/transform/post.py # keep defaults 
            #     "keys": "pred",
            #     "ref_image": "image"
            # },

            # {
            #     "_target_": "EnsureChannelFirstd", # EnsureChannelFirstd(keys="pred"), # https://github.com/Project-MONAI/MONAI/blob/dev/monai/transforms/post/dictionary.py  - do I need to specify the other keys? keep defaults.
            #     "keys": "pred"
            # },
            
            ### RestoreOrientationd is from from lib.transforms.transforms import RestoreOrientationd  ### 
            # RestoreOrientationd(keys="pred", ref_image="image"),
            # Do we need the above?? 
            ### I added SaveImaged below ### 
            {
                "_target_": "SaveImaged",
                "keys": "pred",
                # "meta_keys": "pred_meta_dict",
                "output_dir": "@output_dir"
            }
        ]
    },
    "handlers": [
        {
            "_target_": "CheckpointLoader",
            "load_path": "$@bundle_root + '/models/model.pt'",
            "load_dict": {
                "model": "@network"
            }
        },
        {
            "_target_": "StatsHandler",
            "iteration_log": False
        }
    ],
    "evaluator": {
        "_target_": "SupervisedEvaluator",
        "device": "@device",
        "val_data_loader": "@dataloader",
        "network": "@network",
        "inferer": "@inferer",
        "postprocessing": "@postprocessing",
        "val_handlers": "@handlers",
        "amp": True
    },
    "evaluating": [
        "$setattr(torch.backends.cudnn, 'benchmark', True)",
        "$@evaluator.run()"
    ]
}

In [23]:
inference_filename = os.path.join(config_dir, 'inference.json')
inference_dict["bundle_root"] = main_dir 
inference_dict["dataset_dir"] = data_dir 
config = ConfigParser()
# config.export_config_file(inference_dict, inference_filename, fmt="json")

with open(inference_filename, "w") as outfile: 
  json.dump(inference_dict, outfile, indent=4)
outfile.close()

# Create zip file and download

In [24]:
!zip -r /content/full_ct_segmentation.zip /content/full_ct_segmentation


  adding: content/full_ct_segmentation/ (stored 0%)
  adding: content/full_ct_segmentation/configs/ (stored 0%)
  adding: content/full_ct_segmentation/configs/logging.conf (deflated 51%)
  adding: content/full_ct_segmentation/configs/inference.json (deflated 74%)
  adding: content/full_ct_segmentation/configs/metadata.json (deflated 76%)
  adding: content/full_ct_segmentation/models/ (stored 0%)
  adding: content/full_ct_segmentation/models/model.pt (deflated 7%)


In [25]:
from google.colab import files
files.download("/content/full_ct_segmentation.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run inference (as a check)

In [28]:
!python -m monai.bundle run evaluating \
    --meta_file /content/full_ct_segmentation/configs/metadata.json \
    --config_file /content/full_ct_segmentation/configs/inference.json \
    --logging_file /content/full_ct_segmentation/configs/logging.conf

2023-02-01 11:04:57,402 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-02-01 11:04:57,402 - INFO - > runner_id: 'evaluating'
2023-02-01 11:04:57,402 - INFO - > meta_file: '/content/full_ct_segmentation/configs/metadata.json'
2023-02-01 11:04:57,402 - INFO - > config_file: '/content/full_ct_segmentation/configs/inference.json'
2023-02-01 11:04:57,402 - INFO - > logging_file: '/content/full_ct_segmentation/configs/logging.conf'
2023-02-01 11:04:57,403 - INFO - ---


2023-02-01 11:04:57,403 - INFO - set logging properties based on config: /content/full_ct_segmentation/configs/logging.conf.
2023-02-01 11:04:59,869 - ignite.engine.engine.SupervisedEvaluator - INFO - Engine run resuming from iteration 0, epoch 0 until 1 epochs
2023-02-01 11:04:59,916 - ignite.engine.engine.SupervisedEvaluator - INFO - Restored all variables from /content/full_ct_segmentation/models/model.pt
2023-02-01 11:05:08,721 INFO image_writer.py:194 - writing: /content/full_ct_segmentation/eval/spleen_